In [1]:
import pandas as pd
import numpy as np

In [2]:
n_sequences = 100

In [3]:
targets = pd.read_csv('targets.csv', index_col=0, nrows=n_sequences)

In [4]:
targets = targets.drop(columns=['index', 'opponent'])

In [5]:
sequences = pd.read_csv('sequences.csv', index_col=0, nrows=n_sequences)

**Prepera Data**

Sequences will be transform as follow:


Let sequence be: $[x_1, x_2, x_3, x_4, x_5] \Rightarrow \begin{pmatrix} 
x_1 & 0 & 0 & 0 & 0 \\
0 & x_2 & 0 & 0 & 0 \\
0 & 0 & x_3 & 0 & 0 \\
0 & 0 & 0 & x_4 & 0 \\
0 & 0 & 0 & 0 & x_5 
\end{pmatrix}$

where $x_i \in \{-1, 1\}$ where -1 corresponds to defection and 1 to a cooperation. 

In [6]:
sequences = sequences.replace(to_replace=0, value=-1)

In [7]:
def transform_sequence_to_input_matrix(sequence):
    max_sequence_len = len(sequence)
    data_point = np.zeros((max_sequence_len,
                           max_sequence_len))
    for i in range(max_sequence_len):
        data_point[i - 1][i - 1] = sequence[i]
        
    return data_point

In [8]:
transform_sequence_to_input_matrix([1, -1, 1, 1])

array([[-1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [10]:
max_sequence_len= 204

data = []
for max_length in range(1, max_sequence_len):
    for i, row in sequences.iterrows():
        sequence = row.values[:max_length]
        data.append(transform_sequence_to_input_matrix(sequence))

Validation data are the best responses at time $t + 1$. For example:

Let the sequence be: $[1, -1, 1, -1]$ and let the best response be: [1, -1, -1, -1],

the validation to the transformed sequence:

\begin{pmatrix} 
1 & 0 & 0 & 0  \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0  \\
0 & 0 & 0 & -1 \\
\end{pmatrix} is $$[-1, -1, -1, \dots]$$

In [11]:
# drop gene_0
targets = targets.drop(columns=['gene_0'])

In [12]:
targets.head()

,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,gene_10,...,gene_195,gene_196,gene_197,gene_198,gene_199,gene_200,gene_201,gene_202,gene_203,gene_204
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,0,0
2,0,0,0,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
validation = []
for max_length in range(1, 204):
    for i, row in targets.iterrows():
        validation_point = np.array(row.values[:max_length])
        
        validation.append(validation_point)

**Creating the model**

In [41]:
import tensorflow as tf
import numpy as np

In [43]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout

In [76]:
model = Sequential()

model.add(LSTM(205, return_sequences=True, input_shape=(None, 1)))

model.add(Dropout(rate=0.2))

model.add(TimeDistributed(Dense(1, activation='sigmoid')))

In [73]:
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, None, 205)         169740    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 205)         0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 1)           206       
Total params: 169,946
Trainable params: 169,946
Non-trainable params: 0
_________________________________________________________________


In [61]:
other_data = data[100]

In [64]:
other_data = other_data.reshape(1, other_data.shape[0], other_data.shape[1])

In [65]:
model.predict(other_data)

ValueError: Error when checking input: expected lstm_3_input to have shape (None, 1) but got array with shape (2, 2)

In [14]:
def train_generator():
    while True:
        x = np.random.randint(0, n_sequences)
        test = data[x].reshape((1, data[x].shape[0], data[x].shape[1]))
        y = validation[x].reshape((1, validation[x].shape[0], validation[x].shape[1]))
        
        yield test, y

In [15]:
data[21].reshape((1, data[21].shape[0], data[21].shape[1])).shape

(1, 1, 1)

In [16]:
other_data = np.array([1, 0,]).reshape((1,2,1))

In [17]:
model.predict(other_data).shape

(1, 2, 1)

In [18]:
validation[21].shape

(1, 1)

In [19]:
model.fit_generator(train_generator(), epochs=100, steps_per_epoch=40)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
40/40 [==============================] - 1s 22ms/step - loss: 0.2450 - acc: 0.6500
Epoch 2/100
40/40 [==============================] - 0s 8ms/step - loss: 0.2187 - acc: 0.7750
Epoch 3/100
40/40 [==============================] - 0s 7ms/step - loss: 0.1960 - acc: 0.7750
Epoch 4/100
40/40 [==============================] - 0s 8ms/step - loss: 0.2080 - acc: 0.7000
Epoch 5/100
40/40 [==============================] - 0s 8ms/step - loss: 0.2148 - acc: 0.6750
Epoch 6/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1837 - acc: 0.7250
Epoch 7/100
40/40 [==============================] - 0s 7ms/step - loss: 0.1842 - acc: 0.7500
Epoch 8/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1893 - acc: 0.7500
Epoch 9/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1726 - acc: 0.7750
Epoch 10/100
40/40 [==============================] - 0s 12ms/step - loss: 0.2322 - acc: 0.6250
Epoch 11/

In [31]:
len_dat = 200
data = np.array([0 for _ in range(len_dat)]).reshape((1, len_dat, 1))

In [20]:
other_data = np.array([1, 0, 0 ]).reshape((1, 3, 1))

In [32]:
model.predict(data)

array([[[0.45755786],
        [0.43162292],
        [0.41399622],
        [0.4019867 ],
        [0.39376295],
        [0.38809395],
        [0.3841549 ],
        [0.38139272],
        [0.3794361 ],
        [0.3780352 ],
        [0.3770212 ],
        [0.37627956],
        [0.375732  ],
        [0.37532467],
        [0.37501982],
        [0.37479088],
        [0.3746187 ],
        [0.3744893 ],
        [0.37439236],
        [0.37432015],
        [0.37426665],
        [0.37422746],
        [0.374199  ],
        [0.3741786 ],
        [0.37416425],
        [0.37415433],
        [0.3741476 ],
        [0.3741432 ],
        [0.37414044],
        [0.37413877],
        [0.37413788],
        [0.37413746],
        [0.3741373 ],
        [0.37413734],
        [0.3741375 ],
        [0.37413767],
        [0.37413782],
        [0.37413797],
        [0.37413806],
        [0.37413815],
        [0.3741382 ],
        [0.37413824],
        [0.3741382 ],
        [0.37413818],
        [0.37413818],
        [0

In [35]:
coop_data = np.array([0, 0, 0, 0, 0]).reshape((1, 5, 1))

In [36]:
model.predict(coop_data)

array([[[0.4575579 ],
        [0.43162292],
        [0.4139962 ],
        [0.40198666],
        [0.39376298]]], dtype=float32)

In [24]:
alt_data = coop_data = np.array([1, 0, 1, 0, 1]).reshape((1, 5, 1))

In [25]:
model.predict(alt_data, steps=1)

array([[[0.722763  ],
        [0.61930585],
        [0.7919884 ],
        [0.67818254],
        [0.82229245]]], dtype=float32)

In [26]:
model.predict?